# IMAS Introductory Tutorial

Download this tutorial from the [FuseExamples repository](https://github.com/ProjectTorreyPines/FuseExamples/blob/master/tutorial.ipynb)

## Two packages: IMASdd.jl and IMAS.jl

**IMASdd.jl** is a Julia package that allows manipulating data according to the ITER Modeling and Analysis Suite (IMAS) data schema, also known as the ITER Physics Data Model (PDM) [Imbeaux NF 2015].
Importantly, IMASdd.jl does not use the native IMAS API, but instead implements everything natively in Julia.
https://projecttorreypines.github.io/IMAS.jl/stable/

**IMAS.jl** provides a comprehensive set of mathematical, physics, engineering, and plotting routines. This eliminates the need for individual packages to re-implement these functionalities.
https://projecttorreypines.github.io/IMASdd.jl/stable/

## Basic concepts
IMAS is a standardized data structures for storing experimental and simulation tokamak data
* The interface data structures (IDS) are defined here: https://imas-data-dictionary.readthedocs.io/en/latest/reference_ids.html
* The root of the data structure is `dd`, which stands for "Data Dictionary"  
* IDSs in IMASdd.jl are defined as [nested julia structures](https://raw.githubusercontent.com/ProjectTorreyPines/IMASdd.jl/refs/heads/master/src/dd.jl)

In [ ]:
using Plots
using IMAS # all the functionalities of IMASdd are exposed by the IMAS package

## Basic IMAS usage

In [ ]:
# an empty dd
dd = IMAS.dd();

In [ ]:
# with the following IDSs
keys(dd)

In [ ]:
# view the help for one of the IDSs
help(dd.equilibrium; maxdepth=2)

In [ ]:
# add some data
dd.equilibrium.vacuum_toroidal_field.r0 = 1.7 # a Float64

In [ ]:
# must assign coordinates before data
dd.equilibrium.vacuum_toroidal_field.b0 = [2.0]

In [ ]:
dd.equilibrium.time = [1.0]
dd.equilibrium.vacuum_toroidal_field.b0 = [2.0]

In [ ]:
# IMAS.coordinates() returns what are the coordinates
IMAS.coordinates(dd.equilibrium.vacuum_toroidal_field, :b0)

In [ ]:
# IMAS.units() returns what are the units
IMAS.units(dd.equilibrium.vacuum_toroidal_field, :b0)

## Working with arrays of structures

In [ ]:
# must `resize!()` arrays of structures before using them
resize!(dd.equilibrium.time_slice, 1)

In [ ]:
# Resize with conditions - useful for sources, coils, etc.
source1 = resize!(dd.core_sources.source, "identifier.index" => 1)
source1.identifier.name = "hello"
source2 = resize!(dd.core_sources.source, "identifier.index" => 2)
source1 = resize!(dd.core_sources.source, "identifier.index" => 1)  # Won't add duplicate, but will wipe
dd.core_sources.source

In [ ]:
# by default resize!() operations will wipe the content of IDSs
# however this may not be always desirable. In this case one can use the `wipe=false` keyword argument.
source1 = resize!(dd.core_sources.source, "identifier.index" => 1)
source1.identifier.name = "hello"
source2 = resize!(dd.core_sources.source, "identifier.index" => 2)
source1 = resize!(dd.core_sources.source, "identifier.index" => 1; wipe=false)  # Won't add duplicate, and will not wipte
dd.core_sources.source

In [ ]:
# use classic Julia array functions to manage arrays of structures (empty!, pop!, popat!, deleteat!, push!, pushfirst!, ...)
empty!(dd.core_sources.source)

## Working with time series

The IMAS data structure supports time-dependent data, and IMAS.jl provides ways to handle time data efficiently.

Each `dd` has a `global_time` attribute, which is used throughout FUSE and IMAS to indicate the time at which things should be operate.

In [ ]:
dd.global_time = 1.0

#### Time depedent arrays of structures

In [ ]:
# In addition to the usual `resize!(ids, n::Int)`, time dependent arrays of structures can be resized with:
#   * `resize!(ids, time0)` which will add a time-slice at `time0` seconds
#   * `resize!(ids)` which will add a time-slice at the current global time (this is what you want to use in most cases)

# NOTE: One cannot add new timeslices "in the past". But timeslices can always be added.

resize!(dd.equilibrium.time_slice, 1.0)

In [ ]:
# resize at the dd.global_time
dd.global_time = 2.0
resize!(dd.equilibrium.time_slice)

Here we see that equilibrium has mulitiple time_slices

In [ ]:
dd.equilibrium.time

We can access time-dependent arrays of structures via integer index...

In [ ]:
eqt = dd.equilibrium.time_slice[2]
eqt.time

...or at a given time, by passing the time as a floating point number (in seconds).

NOTE: If we ask a time that is not exactly in the arrays of structures, we'll get the closest (causal!) time-slice

In [ ]:
eqt = dd.equilibrium.time_slice[1.0]
eqt.time

... or at the current `dd.global_time` by leaving the square brackets empty []

**NOTE:** `[]` is what you want to use in most situations that involve time-dependent arrays of structures!

In [ ]:
dd.global_time = 1.0
eqt = dd.equilibrium.time_slice[]
eqt.time

#### Time depedent arrays of data

We can use the `@ddtime()` macro to manipulate these time-dependent arrays at `dd.global_time`.

NOTE: Also in this case, `@ddtime()` will operate on the closest (causal!) time point

NOTE: Use `@ddtime(location_in_dd=value)` to conveniently set the value of a time-dependent vector.

In [ ]:

# this is our current situation
@show dd.equilibrium.time
@show dd.equilibrium.vacuum_toroidal_field.b0;

In [ ]:
dd.global_time = 1.0
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0)

In [ ]:
# if there are more times than data, then the data is assumed to be constant throughout extra times
dd.global_time = 2.0
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0)

In [ ]:
# Add data to an existing time
dd.global_time = 2.0
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0=4.0)

In [ ]:

# this is our current situation
@show dd.equilibrium.time
@show dd.equilibrium.vacuum_toroidal_field.b0;

In [ ]:
# @ddtime uses a causal nearest neighbor interpolation
dd.global_time = 1.7
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0)

In [ ]:
# Add one more time slice, now at a new time
dd.global_time = 3.0
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0=6.0)

In [ ]:

# this is our current situation
@show dd.equilibrium.time
@show dd.equilibrium.vacuum_toroidal_field.b0;

In [ ]:
# @ddtime uses a causal nearest neighbor interpolation also for forward extrapolation
dd.global_time = 100.0
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0)

In [ ]:
# @ddtime does not interpolate back in time
dd.global_time = 0.0
@ddtime(dd.equilibrium.vacuum_toroidal_field.b0)

## Loading IMAS data

Load IMAS data from JSON. Other formats are available: https://projecttorreypines.github.io/IMASdd.jl/stable/api/#IO

In [ ]:
dd = IMAS.json2imas("D3D_147131.json");

## Exploring the data dictionary

Display part of the equilibrium data in `dd`

In [ ]:
dd.equilibrium.time_slice[1].boundary

this can be done up to a certain depth with `print_tree`

In [ ]:
print_tree(dd.equilibrium.time_slice[1].boundary; maxdepth=1)

## Plotting data from `dd`
FUSE uses `Plots.jl` recipes for visualizing data from `dd`.

This allows different plots to be shown when calling `plot()` on different items in the data structure.

Learn more about Plots.jl [here](https://docs.juliaplots.org)

For example plotting the equilibrium...

In [ ]:
plot(dd.equilibrium)

Whant to know what arguments can be passed? use `help_plot()` function

In [ ]:
help_plot(dd.equilibrium; core_profiles_overlay=true, levels_in=21, levels_out=5, show_secondary_separatrix=true, coordinate=:rho_tor_norm)

These plots can be composed by calling `plot!()` instead of `plot()`

In [ ]:
plot(dd.equilibrium; color=:gray, cx=true)
plot!(dd.build.layer)
plot!(dd.pf_active)
plot!(dd.pf_passive)

Try plotting the other IDSs, like `core_profiles`

In [ ]:
plot(dd.core_profiles)

plot time dependent data using the [Interact.jl](https://github.com/JuliaGizmos/Interact.jl) package

In [ ]:
using Interact
@manipulate for time0 in dd.equilibrium.time
    plot(dd.equilibrium; cx=true, time0)
    plot!(dd.wall)
    plot!(dd.pf_active; time0)
    plot!(dd.pf_passive; time0)
    plot!(size=(600,600))
end

Plotting an array...

In [ ]:
plot(dd.core_profiles.profiles_1d[].pressure_thermal)

...is different from plotting a field from the IDS (which plots the quantity against its coordinate and with units)

In [ ]:
plot(dd.core_profiles.profiles_1d[], :pressure_thermal)

Customizing plot attributes:

In [ ]:
plot(dd.core_profiles.profiles_1d[], :pressure_thermal; label="", linewidth=2, color=:red, labelfontsize=25)

Use `findall(ids, r"...")` to search for certain fields. In Julia, string starting with `r` are regular expressions.

In [ ]:
findall(dd.equilibrium.time_slice[], r"\.psi")

`findall(ids, r"...")` can be combined with `plot()` to plot multiple fields

In [ ]:
plot(findall(dd.equilibrium.time_slice[], r"\.psi"))

## Expressions in `dd`

Some fields in the data dictionary are expressions (ie. Functions).
For example `dd.core_profiles.profiles_1d[].pressure` is dynamically calculated as the product of thermal densities and temperature with addition of fast ions contributions

In [ ]:
dd.global_time = 1.0
print_tree(dd.core_profiles.profiles_1d[]; maxdepth=1)

accessing a dynamic expression, automatically evaluates it

In [ ]:
dd.core_profiles.profiles_1d[].conductivity_parallel

In addition to evaluating expressions by accessing them, expressions in the tree can be evaluated using `IMAS.freeze(ids)`

NOTE: `IMAS.freeze(ids, field::Symbol)` works on a single field and `IMAS.refreeze!(ids, field)` forces re-evaluation of an expression. Also, `IMAS.empty!(ids, field::Symbol)` can be used to revert a frozen field back into an expression.

In [ ]:
print_tree(IMAS.freeze(dd.core_profiles.profiles_1d[]); maxdepth=1)

## Comparing two IDSs
We can introduce a change in the `dd1` and spot it with the `diff` function

In [ ]:
dd1 = deepcopy(dd)
dd1.equilibrium.time_slice[].global_quantities.ip = -100.0
IMAS.diff(dd.equilibrium, dd1.equilibrium; verbose=true);

## Summary
Snapshot of `dd` in 0D quantities (evaluated at `dd.global_time`).

In [ ]:
IMAS.extract(dd)

## Physics functions

IMAS.jl provides [an extensive collection of physics functions](https://projecttorreypines.github.io/IMAS.jl/dev/api/) that work directly on the `dd`. These functions span the breadth and depth of tokamak plasma modeling, covering everything from basic geometry calculations and flux surface tracing to advanced multi-physics simulations including flux surface and field lines tracing, transport analysis, heating and current drive modeling, nuclear fusion reactions, radiation physics, and comprehensive plasma-material interaction studies.

By offering validated, high-performance implementations of these core plasma physics routines, IMAS.jl enables the fusion research community to build upon a common foundation rather than reinventing fundamental algorithms, thereby accelerating research progress and ensuring consistency across diverse simulation workflows.

In [ ]:
# example: Field line tracing

In [ ]:
eqt = dd.equilibrium.time_slice[]

r = 2.3457
z = 0.1
@time t1 = IMAS.trace_field_line(eqt,r,z; max_turns=100,step_size=0.1);
@time t2 = IMAS.trace_field_line(eqt,r,z; max_turns=100,step_size=-0.1);

plot(eqt; cx=true, primary=false)
plot!(t1.r,t1.z; label="Upwind")
plot!(t2.r,t2.z; label="Downwind")
display(scatter!([r],[z]; label="Starting point", color=:black))

plot3d(t1.x,t1.y,t1.z;label="Upwind")
plot3d!(t2.x,t2.y,t2.z; label="Downwind")
plot3d!(; xlabel=:x, ylabel=:y, zlabel=:z, aspect_ratio=:equal, size=(600,600),camera=(45, 60),)